# Imports

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
import ta

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import f1_score
from imblearn.under_sampling import RandomUnderSampler
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from pprint import pprint

import gc
import optuna
from sklearn.feature_selection import RFE


/tmp/ipykernel_20753/384002432.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('Data/Training/pair_features_1_pairs1_300_120.csv')
df.head()

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,AMAT,2001-02-09 00:00:00,22.531,21.000,32504200.000,15.688,BLK,43.000,42.200,27200.000,26.388,10.699,12.463,5.885,10.699,NaN,0.000,10.937,0.000
1,AMAT,2001-02-12 00:00:00,22.500,21.102,28449600.000,16.194,BLK,43.710,42.000,33100.000,27.132,10.937,12.472,5.881,10.937,NaN,0.000,12.081,0.000
2,AMAT,2001-02-13 00:00:00,22.500,20.500,42626000.000,15.182,BLK,43.990,43.140,33900.000,27.263,12.081,12.481,5.879,12.081,NaN,0.000,9.721,0.000
3,AMAT,2001-02-14 00:00:00,23.844,21.000,95765800.000,17.229,BLK,43.460,42.500,20800.000,26.950,9.721,12.484,5.877,9.721,NaN,0.000,8.942,0.000
4,AMAT,2001-02-15 00:00:00,25.312,23.719,55611800.000,18.196,BLK,43.500,42.700,18400.000,27.138,8.942,12.481,5.879,8.942,NaN,0.000,8.572,0.000


In [3]:
df['pnls'] = df['pnls']*100.0
# df = df[df.pnls != 0]

In [4]:
total_days = df.Date.nunique()
total_days

5757

In [5]:
# df.dropna()

In [6]:
df.pnls.describe()

count   5757.000
mean       0.000
std        0.000
min        0.000
25%        0.000
50%        0.000
75%        0.000
max        0.000
Name: pnls, dtype: float64

## Technical Indicators

### Load SPY

In [7]:
# df = df.dropna()
df

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,AMAT,2001-02-09 00:00:00,22.531,21.000,32504200.000,15.688,BLK,43.000,42.200,27200.000,26.388,10.699,12.463,5.885,10.699,NaN,0.000,10.937,0.000
1,AMAT,2001-02-12 00:00:00,22.500,21.102,28449600.000,16.194,BLK,43.710,42.000,33100.000,27.132,10.937,12.472,5.881,10.937,NaN,0.000,12.081,0.000
2,AMAT,2001-02-13 00:00:00,22.500,20.500,42626000.000,15.182,BLK,43.990,43.140,33900.000,27.263,12.081,12.481,5.879,12.081,NaN,0.000,9.721,0.000
3,AMAT,2001-02-14 00:00:00,23.844,21.000,95765800.000,17.229,BLK,43.460,42.500,20800.000,26.950,9.721,12.484,5.877,9.721,NaN,0.000,8.942,0.000
4,AMAT,2001-02-15 00:00:00,25.312,23.719,55611800.000,18.196,BLK,43.500,42.700,18400.000,27.138,8.942,12.481,5.879,8.942,NaN,0.000,8.572,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,AMAT,2023-12-21 00:00:00,161.820,159.580,4257300.000,161.117,BLK,799.370,791.660,341100.000,799.100,637.983,551.310,42.442,637.983,NaN,0.000,640.644,0.000
5753,AMAT,2023-12-22 00:00:00,163.000,160.840,2770600.000,161.776,BLK,807.520,800.150,426300.000,802.420,640.644,551.885,42.479,640.644,NaN,0.000,642.608,0.000
5754,AMAT,2023-12-26 00:00:00,164.970,162.100,2520500.000,164.002,BLK,809.100,803.440,280600.000,806.610,642.608,552.500,42.451,642.608,NaN,0.000,649.318,0.000
5755,AMAT,2023-12-27 00:00:00,164.990,163.530,3319600.000,163.932,BLK,813.250,808.630,298000.000,813.250,649.318,553.078,42.587,649.318,NaN,0.000,651.566,0.000


### Beta

In [8]:
def compute_beta(rs,df,ticker,market):
    rows = df.loc[rs.index]
    sec_returns = np.log( df[[ticker,market]] / df[[ticker,market]].shift(1) ) 

    cov = sec_returns.cov() * 250
    cov_with_market = cov.iloc[0,1]
    market_var = sec_returns[market].var() * 250
    beta = cov_with_market / market_var
    return beta

In [9]:
all_tickers = df['Ticker_P1'].unique().tolist()+df['Ticker_P2'].unique().tolist()
len(all_tickers)

2

### Single Ticker Features

In [10]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread',
       'actual_abs_spread_std'],
      dtype='object')

In [11]:
df.sort_values('Date')

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,AMAT,2001-02-09 00:00:00,22.531,21.000,32504200.000,15.688,BLK,43.000,42.200,27200.000,26.388,10.699,12.463,5.885,10.699,NaN,0.000,10.937,0.000
1,AMAT,2001-02-12 00:00:00,22.500,21.102,28449600.000,16.194,BLK,43.710,42.000,33100.000,27.132,10.937,12.472,5.881,10.937,NaN,0.000,12.081,0.000
2,AMAT,2001-02-13 00:00:00,22.500,20.500,42626000.000,15.182,BLK,43.990,43.140,33900.000,27.263,12.081,12.481,5.879,12.081,NaN,0.000,9.721,0.000
3,AMAT,2001-02-14 00:00:00,23.844,21.000,95765800.000,17.229,BLK,43.460,42.500,20800.000,26.950,9.721,12.484,5.877,9.721,NaN,0.000,8.942,0.000
4,AMAT,2001-02-15 00:00:00,25.312,23.719,55611800.000,18.196,BLK,43.500,42.700,18400.000,27.138,8.942,12.481,5.879,8.942,NaN,0.000,8.572,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,AMAT,2023-12-21 00:00:00,161.820,159.580,4257300.000,161.117,BLK,799.370,791.660,341100.000,799.100,637.983,551.310,42.442,637.983,NaN,0.000,640.644,0.000
5753,AMAT,2023-12-22 00:00:00,163.000,160.840,2770600.000,161.776,BLK,807.520,800.150,426300.000,802.420,640.644,551.885,42.479,640.644,NaN,0.000,642.608,0.000
5754,AMAT,2023-12-26 00:00:00,164.970,162.100,2520500.000,164.002,BLK,809.100,803.440,280600.000,806.610,642.608,552.500,42.451,642.608,NaN,0.000,649.318,0.000
5755,AMAT,2023-12-27 00:00:00,164.990,163.530,3319600.000,163.932,BLK,813.250,808.630,298000.000,813.250,649.318,553.078,42.587,649.318,NaN,0.000,651.566,0.000


In [12]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread',
       'actual_abs_spread_std'],
      dtype='object')

In [13]:
label = 'actual_abs_spread'
exclude = ['pnls', 'High_P1', 'High_P2', 'Low_P1', 'Low_P2', 'abs_spread', 'abs_spread_mean', 'abs_spread_std', 'abs_spread_std_MA', 'actual_abs_spread_std']
X = df
y = df[label]

X = X.drop(['Date', 'Ticker_P1', 'Ticker_P2', 'Volume_P1', 'Volume_P2', label],axis=1)
X = X.drop(exclude, axis=1)

In [14]:
X = X.dropna()
X

,Close_P1,Close_P2,abs_spread_mean_MA
0,15.688,26.388,10.699
1,16.194,27.132,10.937
2,15.182,27.263,12.081
3,17.229,26.950,9.721
4,18.196,27.138,8.942
...,...,...,...
5752,161.117,799.100,637.983
5753,161.776,802.420,640.644
5754,164.002,806.610,642.608
5755,163.932,813.250,649.318


## Train test split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42,shuffle=False)

In [16]:
print(df.loc[X_train.index].Date.min(),df.loc[X_train.index].Date.max())

2001-02-09 00:00:00 2019-06-03 00:00:00


In [17]:
print(df.loc[X_val.index].Date.min(),df.loc[X_val.index].Date.max())

2019-06-04 00:00:00 2021-09-14 00:00:00


In [18]:
print(df.loc[X_test.index].Date.min(),df.loc[X_test.index].Date.max())

2021-09-15 00:00:00 2023-12-28 00:00:00


## LSTM Model

In [19]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
#Creating the LSTM model
class lstm(Model):
    def __init__(self, in_shape):
        super(lstm, self).__init__()
        self.shape = in_shape
        self.lstm = LSTM(1024, input_shape=self.shape, dropout=0.2, return_sequences = True)
        self.dropout = Dropout(0.2)
        self.dense = Dense(1)

    def call(self, x):
        x = self.lstm(x)
#         x = self.dropout(x)
        x = self.dense(x)
        return x

2024-03-20 03:45:32.383809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 03:45:33.126310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [20]:
# Reshape X for model training
trainX = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)).astype(np.float32)
valX = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1)).astype(np.float32)
testX = np.reshape(X_test, ( X_test.shape[0], X_test.shape[1], 1)).astype(np.float32)

In [21]:
trainX.shape

(4605, 3, 1)

In [22]:
#initializing the lstm model
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=5,\
                                            min_delta=0.001,restore_best_weights=True,verbose=1)

lstm_model = lstm((trainX.shape[0], trainX.shape[1]))
lstm_model.compile(loss='mae', 
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), metrics=['mae'])
#fitting the model
history = lstm_model.fit(
                    trainX, 
                    y_train,
                    validation_data=(valX, y_val), 
                    epochs=100, 
                    batch_size=32,
                    verbose=1, 
                    callbacks=callback,
                    shuffle=False)       

Epoch 1/100


2024-03-20 03:45:33.955747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


144/144 [==============================] - 15s 87ms/step - loss: 123.8339 - mae: 123.8339 - val_loss: 441.0285 - val_mae: 441.0285
Epoch 2/100
144/144 [==============================] - 12s 84ms/step - loss: 112.0600 - mae: 112.0600 - val_loss: 420.6039 - val_mae: 420.6039
Epoch 3/100
144/144 [==============================] - 12s 84ms/step - loss: 100.6540 - mae: 100.6540 - val_loss: 399.1995 - val_mae: 399.1995
Epoch 4/100
144/144 [==============================] - 12s 84ms/step - loss: 87.6706 - mae: 87.6706 - val_loss: 378.3969 - val_mae: 378.3969
Epoch 5/100
144/144 [==============================] - 12s 84ms/step - loss: 79.6302 - mae: 79.6302 - val_loss: 361.3925 - val_mae: 361.3925
Epoch 6/100
 36/144 [======>.......................] - ETA: 9s - loss: 17.2343 - mae: 17.2343

KeyboardInterrupt: 

In [ ]:
lstm_model.summary()

In [ ]:
# Plot line graph to show amount loss according the the epoch
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss Plot -- LSTM', fontsize=14, color="black", fontweight="bold")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
inference_result = lstm_model.predict(testX).flatten()

In [ ]:
y_test.plot.hist(bins=12, alpha=0.5)
y_test.describe()

In [ ]:
pd.DataFrame(inference_result).plot.hist(bins=12, alpha=0.5)
pd.Series(inference_result).describe()

In [ ]:
lstm_mae = (inference_result[:len(y_test)] - y_test).abs().mean()
lstm_mae

In [ ]:
inference_result

In [ ]:
mae = lstm_mae / y_test.abs().mean()
mae

In [ ]:
# Normality test

from scipy.stats import shapiro
# normality test
stat, p = shapiro(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# D’Agostino’s K^2 Test
from scipy.stats import normaltest
# normality test
stat, p = normaltest(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# Anderson-Darling Test
from scipy.stats import anderson
result = anderson(inference_result)
print('Statistic: %.3f' % result.statistic)
p = 0
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
if result.statistic < result.critical_values[i]:
    print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
else:
    print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))